# Multistep pipeline for spot (pair imaging)

Here, we extend to the overview-detail strategy for FISH spot (pair) imaging with an intermediate step.

After acquiring an onverview (at low resolution), a small secondary confocal image of just a few microns FOV size will be acquired around groups of detected spots. Finally, we detect spot pairs again and acquire only very small (MINFIELD-range of less than a micron) images around the detections with STED.

Furthermore, we use **multiple settings files** at the highest resolution level to acquire multiple image configrations for each detection. This way, we can acquire both an image with 2D depletion and an image with 3D depletion for each detection. 

In [3]:
from matplotlib import pyplot as plt

from autosted import AcquisitionPipeline
from autosted.callback_buildingblocks import (
    BoundingBoxLocationGrouper,
    FOVSettingsGenerator,
    JSONSettingsLoader,
    LocationKeeper,
    LocationRemover,
    NewestDataSelector,
    NewestSettingsSelector,
    ScanModeSettingsGenerator,
    ScanOffsetsSettingsGenerator,
    SpiralOffsetGenerator,
    StageOffsetsSettingsGenerator,
)
from autosted.detection import SimpleFocusPlaneDetector
from autosted.detection.legacy import LegacySpotPairFinder
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import TimedStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator

# import specpy as sp

# plotting params
plt.rcParams["figure.figsize"] = [7, 7]

### Parameters

In [10]:
save_path = "acquisition_data/threestep_spots"

# time to image in seconds
time_to_image = 1 * 60 * 60

# fov sizes and pixel sizes at various levels
# NOTE: may be set to None to just use values from settings file
fov_sizes = {
    "overview": [2e-6, 50e-6, 50e-6],
    "detail": [2e-6, 2e-6, 2e-6],
    "sted": [5e-7, 5e-7, 5e-7],
}
pixel_sizes = {
    "overview": [5e-7, 1.5e-7, 1.5e-7],
    "detail": [1e-7, 0.6e-7, 0.6e-7],
    "sted": [30e-9, 10e-9, 10e-9],
}

# how far to move in overview spiral (set larger than fov_size to have non-overlapping)
overview_spiral_move = [55e-6, 55e-6]

# 2-channel settings for levels overview + detail
overview_settings_file = "config_json/multistep_spots_confocal_590_640.json"
detail_settings_file = "config_json/multistep_spots_confocal_590_640.json"

# 2 STED settings for 2D and 3D sted
sted_settings_files = [
    "config_json/multistep_spots_sted_590_640_2d.json",
    "config_json/multistep_spots_sted_590_640_3d.json",
]

overview_detector_kwargs = {"sigma": 2, "thresholds": [0.4, 0.4]}
detail_detector_kwargs = {"sigma": 3, "thresholds": [0.2, 0.2], "median_radius": 10}

### Pipeline run

The actual pipeline setup is similar to a two-level version. One small detail: we wrap the detector at the intermediate detail level in a ```BoundingBoxLocationGrouper```: this groups detections into non-overlapping groups and ensures that each detected spot is imaged only once. 

In [ ]:
# init 3-level pipeline
pl = AcquisitionPipeline(
    save_path, ("overview", "detail", "sted"), save_combined_hdf5=True
)

# overview task generator: traverse in spiral with autofocus
atg_overview = AcquisitionTaskGenerator(
    "overview",
    LocationRemover(JSONSettingsLoader(overview_settings_file)),
    SpiralOffsetGenerator(overview_spiral_move, get_current_stage_coords()),
    ScanModeSettingsGenerator("xyz"),
    FOVSettingsGenerator(fov_sizes["overview"], pixel_sizes["overview"]),
    StageOffsetsSettingsGenerator(
        SimpleFocusPlaneDetector(NewestDataSelector(pl, "overview"))
    ),
)

# spot pair detector in overview
detector = LegacySpotPairFinder(
    NewestDataSelector(pl, "overview"), **overview_detector_kwargs, plot_detections=True
)

# detail task generation: settings from last overview + new FOV + grouped detections
atg_detail = AcquisitionTaskGenerator(
    "detail",
    LocationRemover(JSONSettingsLoader(detail_settings_file)),
    LocationKeeper(NewestSettingsSelector(pl, "overview")),
    ScanModeSettingsGenerator("xyz"),
    FOVSettingsGenerator(fov_sizes["detail"], pixel_sizes["detail"]),
    ScanOffsetsSettingsGenerator(
        BoundingBoxLocationGrouper(detector, fov_sizes["detail"])
    ),
)

# spot pair detector in detail
detector_detail = LegacySpotPairFinder(
    NewestDataSelector(pl, "detail"), **detail_detector_kwargs, plot_detections=True
)

# super-resolution detail task generation: settings from last detail + new FOV
atg_detail_more = AcquisitionTaskGenerator(
    "sted",
    # NOTE: as we have multiple configuration files, we will generate multiple configurations for each detection
    LocationRemover(JSONSettingsLoader(sted_settings_files, None)),
    LocationKeeper(NewestSettingsSelector(pl, "detail")),
    ScanModeSettingsGenerator("xyz", False),
    FOVSettingsGenerator(fov_sizes["sted"], pixel_sizes["sted"]),
    ScanOffsetsSettingsGenerator(detector_detail),
)

pl.add_stopping_condition(TimedStoppingCriterion(time_to_image))

# overview callback: re-add overview
pl.add_callback(atg_overview, "overview")
# overview callback 2: detect & do detail
pl.add_callback(atg_detail, "overview")
# detail callback: detect & do more detail
pl.add_callback(atg_detail_more, "detail")

# GO
pl.run(atg_overview)

## Image channels independently + orthogonal cuts

When imaging tiny fields in multiple channels, the separation between the actual objects might limit how small we can go. Therefore, we can image both channels independently, centered around the respective spot.

Furthermore, instead of acquiring stacks, we could also just image an xy and xz cut through the middle of the spot-like signal.

Thus instead of imaging two stacks around the detected spot pair, we acquire 4 orthogonal cuts:

* xy cut in channel 1 with 2D depletion
* xy cut in channel 2 with 2D depletion
* xz cut in channel 1 with 3D depletion
* xz cut in channel 2 with 3D depletion

### Parameters

In [4]:
save_path = "acquisition_data/threestep_spots_orthogonal_cuts_3"

# time to image in seconds
time_to_image = 1 * 60 * 60

# fov sizes and pixel sizes at various levels
# NOTE: may be set to None to just use values from settings file
fov_sizes = {
    "overview": [2e-6, 5e-5, 5e-5],
    "detail": [2e-6, 2e-6, 2e-6],
    "sted": [5e-7, 5e-7, 5e-7],
}
pixel_sizes = {
    "overview": [5e-7, 1.5e-7, 1.5e-7],
    "detail": [1e-7, 0.6e-7, 0.6e-7],
    "sted": [10e-9, 10e-9, 10e-9],
}

# how far to move in overview spiral (set larger than fov_size to have non-overlapping)
overview_spiral_move = [55e-6, 55e-6]

# 2-channel settings for levels overview + detail
overview_settings_file = "config_json/multistep_spots_confocal_590_640.json"
detail_settings_file = "config_json/multistep_spots_confocal_590_640.json"

# 4 single-channel settings for orthogonal cuts in 2 channels
sted_settings_files = [
    "config_json/multistep_spots_sted_590_2d.json",
    "config_json/multistep_spots_sted_640_2d.json",
    "config_json/multistep_spots_sted_590_3d.json",
    "config_json/multistep_spots_sted_640_3d.json",
]

overview_detector_kwargs = {"sigma": 2, "thresholds": [0.4, 0.4]}
detail_detector_kwargs = {"sigma": 3, "thresholds": [0.2, 0.2], "median_radius": 10}

### Run pipeline

The pipeline is very similar to above, but we make a few changes:

* we use a ```PairedLegacySpotPairFinder``` that detects spot pairs but returns both coordinates
* this is the wrapped in a ```ResultsRepeater``` and a ```MultipleScanOffsetsSettingsGenerator``` to repeat the results twice, to match the 4 configurations

In [ ]:
from autosted.detection.legacy import PairedLegacySpotPairFinder
from autosted.callback_buildingblocks import (
    ResultsRepeater,
    MultipleScanOffsetsSettingsGenerator,
)

# init 3-level pipeline
pl = AcquisitionPipeline(
    save_path, ("overview", "detail", "sted"), save_combined_hdf5=True
)

# overview task generator: traverse in spiral with autofocus
atg_overview = AcquisitionTaskGenerator(
    "overview",
    LocationRemover(JSONSettingsLoader(overview_settings_file)),
    SpiralOffsetGenerator(overview_spiral_move, get_current_stage_coords()),
    ScanModeSettingsGenerator("xyz"),
    FOVSettingsGenerator(fov_sizes["overview"], pixel_sizes["overview"]),
    StageOffsetsSettingsGenerator(
        SimpleFocusPlaneDetector(NewestDataSelector(pl, "overview"))
    ),
)

# spot pair detector in overview
detector = LegacySpotPairFinder(
    NewestDataSelector(pl, "overview"), **overview_detector_kwargs, plot_detections=True
)

# detail task generation: settings from last overview + new FOV + grouped detections
atg_detail = AcquisitionTaskGenerator(
    "detail",
    LocationRemover(JSONSettingsLoader(detail_settings_file)),
    LocationKeeper(NewestSettingsSelector(pl, "overview")),
    ScanModeSettingsGenerator("xyz"),
    FOVSettingsGenerator(fov_sizes["detail"], pixel_sizes["detail"]),
    ScanOffsetsSettingsGenerator(
        BoundingBoxLocationGrouper(detector, fov_sizes["detail"])
    ),
)

# spot pair detector in detail
# NOTE: we use PairedLegacySpotPairFinder, which will return the coordinates of both detected spots
detector_detail = PairedLegacySpotPairFinder(
    NewestDataSelector(pl, "detail"), **detail_detector_kwargs, plot_detections=True
)

# super-resolution detail task generation: settings from last detail + new FOV
atg_detail_more = AcquisitionTaskGenerator(
    "sted",
    LocationRemover(
        JSONSettingsLoader(sted_settings_files, None, as_measurements=False)
    ),
    LocationKeeper(NewestSettingsSelector(pl, "detail")),
    # NOTE: 4 scan mode configurations (each orthogonal cut twice for different channels)
    ScanModeSettingsGenerator(
        ["xy", "xy", "xz", "xz"],
        as_measurements=False,
    ),
    FOVSettingsGenerator(fov_sizes["sted"], pixel_sizes["sted"]),
    # get 2x2 scan offsets (multiple configurations) based on detections
    # NOTE: we repeat detected pairs (coords_ch1, coords_ch2) 2 times to get (coords_ch1, coords_ch2, coords_ch1, coords_ch2)
    MultipleScanOffsetsSettingsGenerator(
        ResultsRepeater(detector_detail, 2, True, True)
    ),
)

pl.add_stopping_condition(TimedStoppingCriterion(time_to_image))

# overview callback: re-add overview
pl.add_callback(atg_overview, "overview")
# overview callback 2: detect & do detail
pl.add_callback(atg_detail, "overview")
# detail callback: detect & do more detail
pl.add_callback(atg_detail_more, "detail")

# GO
pl.run(atg_overview)